<a href="https://colab.research.google.com/github/ifymifan/IOT-AI/blob/main/LAB_5_1_Spark_MLlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install findspark
import findspark
import pyspark # this was also missing in the original code
findspark.init()
findspark.find()
# import findsoark # typo in the original code. I'm assuming the user meant to type findspark?
findspark.find() # if user did in fact mean findsoark, then user should !pip install findsoark and import findsoark

'/usr/local/lib/python3.11/dist-packages/pyspark'

In [12]:
from pyspark.sql import SparkSession

from google.colab import drive
drive.mount('/content/drive')

# Create a spark Session

spark =  SparkSession.builder\
        .master("local")\
        .appName("Titanic data")\
        .getOrCreate()

# Check spark session information
spark # Changed 'Spark' to 'spark' to access the SparkSession object

Mounted at /content/drive


In [13]:
from pyspark.sql import SparkSession

In [25]:
df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .load("/content/drive/MyDrive/titanic/train.csv")

In [26]:
from pyspark.sql.functions import col

In [27]:
dataset = df.select(col('Survived').cast('float'),
                         col('Pclass').cast('float'),
                         col('Sex'),
                         col('Age').cast('float'),
                         col('Fare').cast('float'),
                         col('Embarked')
                        )

In [28]:
dataset.show()

+--------+------+------+----+-------+--------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|
+--------+------+------+----+-------+--------+
|     0.0|   3.0|  male|22.0|   7.25|       S|
|     1.0|   1.0|female|38.0|71.2833|       C|
|     1.0|   3.0|female|26.0|  7.925|       S|
|     1.0|   1.0|female|35.0|   53.1|       S|
|     0.0|   3.0|  male|35.0|   8.05|       S|
|     0.0|   3.0|  male|NULL| 8.4583|       Q|
|     0.0|   1.0|  male|54.0|51.8625|       S|
|     0.0|   3.0|  male| 2.0| 21.075|       S|
|     1.0|   3.0|female|27.0|11.1333|       S|
|     1.0|   2.0|female|14.0|30.0708|       C|
|     1.0|   3.0|female| 4.0|   16.7|       S|
|     1.0|   1.0|female|58.0|  26.55|       S|
|     0.0|   3.0|  male|20.0|   8.05|       S|
|     0.0|   3.0|  male|39.0| 31.275|       S|
|     0.0|   3.0|female|14.0| 7.8542|       S|
|     1.0|   2.0|female|55.0|   16.0|       S|
|     0.0|   3.0|  male| 2.0| 29.125|       Q|
|     1.0|   2.0|  male|NULL|   13.0|       S|
|     0.0|   

In [29]:
from pyspark.sql.functions import isnull, when, count, col

In [30]:
dataset.select([count(when(isnull(c), c)).alias(c) for c in dataset.columns]).show()

+--------+------+---+---+----+--------+
|Survived|Pclass|Sex|Age|Fare|Embarked|
+--------+------+---+---+----+--------+
|       0|     0|  0|177|   0|       2|
+--------+------+---+---+----+--------+



In [34]:
dataset = dataset.replace('?',None)\
.dropna(how='any')
#

In [35]:
dataset.select([count(when(isnull(c), c)).alias(c) for c in dataset.columns]).show()

+--------+------+---+---+----+--------+
|Survived|Pclass|Sex|Age|Fare|Embarked|
+--------+------+---+---+----+--------+
|       0|     0|  0|  0|   0|       0|
+--------+------+---+---+----+--------+



In [36]:
dataset.show(3)

+--------+------+------+----+-------+--------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|
+--------+------+------+----+-------+--------+
|     0.0|   3.0|  male|22.0|   7.25|       S|
|     1.0|   1.0|female|38.0|71.2833|       C|
|     1.0|   3.0|female|26.0|  7.925|       S|
+--------+------+------+----+-------+--------+
only showing top 3 rows



In [39]:
from pyspark.ml.feature import StringIndexer # Changed 'mi' to 'ml' in the import statement

In [40]:
dataset = StringIndexer(
    inputCol='Sex',
    outputCol='Gender',
    handleInvalid='keep').fit(dataset).transform(dataset)

In [41]:
dataset.show(2)

+--------+------+------+----+-------+--------+------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|Gender|
+--------+------+------+----+-------+--------+------+
|     0.0|   3.0|  male|22.0|   7.25|       S|   0.0|
|     1.0|   1.0|female|38.0|71.2833|       C|   1.0|
+--------+------+------+----+-------+--------+------+
only showing top 2 rows



In [42]:
#Drop unnecessary comums
dataset = dataset.drop('Sex')
dataset = dataset.drop('Embarked')


In [48]:
#Assembly all the feautres with vectorAssembly
from pyspark.ml.feature import VectorAssembler
required_features = ['Pclass',
                    'Age',
                    'Fare',
                    'Gender',
                     'Survived'] # Changed 'Board' to 'Survived' as 'Board' column doesn't exist
Assembly = VectorAssembler(inputCols=required_features,outputCol='features')
# Check if 'features' column already exists
if 'features' not in dataset.columns:
    # If not, apply the transformation
    dataset = Assembly.transform(dataset) # Use the correct variable name 'Assembly'
else:
    print("Features column already exists.")
# %%
dataset.show(5) # Changed 'transfermed_data' to 'dataset' to display the transformed data

Features column already exists.
+--------+------+----+-------+------+--------------------+
|Survived|Pclass| Age|   Fare|Gender|            features|
+--------+------+----+-------+------+--------------------+
|     0.0|   3.0|22.0|   7.25|   0.0|[3.0,22.0,7.25,0....|
|     1.0|   1.0|38.0|71.2833|   1.0|[1.0,38.0,71.2833...|
|     1.0|   3.0|26.0|  7.925|   1.0|[3.0,26.0,7.92500...|
|     1.0|   1.0|35.0|   53.1|   1.0|[1.0,35.0,53.0999...|
|     0.0|   3.0|35.0|   8.05|   0.0|[3.0,35.0,8.05000...|
+--------+------+----+-------+------+--------------------+
only showing top 5 rows



In [50]:
# Splitting dataset into train and test
(training_data, test_data) = dataset.randomSplit([0.8,0.2]) # Changed 'transfermed_data' to 'dataset'
print("Number of train samplea: " + str(training_data.count()))
print("Number of test samplea: " + str(test_data.count()))

Number of train samplea: 571
Number of test samplea: 141


In [51]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol='Survived',
                            featuresCol='features',
                            maxDepth=5)

In [52]:
rf = pyspark.ml.classification.RandomForestClassifier(labelCol='Survived',
                            featuresCol='features',
                            maxDepth=5)


In [54]:
model = rf.fit(training_data)
predictions = model.transform(test_data) # Changed 'transformation' to 'transform' to get predictions

In [71]:
#Evaluation
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol='Survived',
    predictionCol='prediction',
    metricName='accuracy')

In [73]:
accuracy = evaluator.evaluate(predictions) # Use 'predictions' instead of 'predictions_test'
print('Training Accuracy = ' , accuracy)

Training Accuracy =  1.0
